In [6]:
import pandas as pd
import pyomo.environ as pyo
from omlt import OmltBlock
from omlt.neuralnet import ReluBigMFormulation
from omlt.io import load_keras_sequential
import tensorflow
import pandas as pd
import pickle

In [7]:
dict_scalers = pickle.load(open("scalers.pkl", "rb"))
x_offset = dict_scalers["x_offset"]
x_factor = dict_scalers["x_factor"]
y_offset = dict_scalers["y_offset"]
y_factor = dict_scalers["y_factor"]
scaled_lb = dict_scalers["scaled_lb"]
scaled_ub = dict_scalers["scaled_ub"]
inputs = ["x", "y"]
outputs = ["f1", "f2"]
# 1. Charger le modèle Keras dans un format compatible avec OMLT
data = pd.read_parquet("../data/data.parquet")
X = data[['x', 'y']]
y = data[['f1', 'f2']]
x_max = X['x'].max()
y_max = X['y'].max()

In [8]:
from omlt.neuralnet import FullSpaceSmoothNNFormulation

# Charger le modèle Keras
nn = tensorflow.keras.models.load_model("plane_nn_sigmoid.keras", compile=False)
net = load_keras_sequential(nn, unscaled_input_bounds={0: (0,5 ), 1: (0,5 )})

# 2. Définir le modèle Pyomo pour l'optimisation
pyomo_model = pyo.ConcreteModel()

# Ajouter un bloc OMLT pour intégrer le réseau de neurones
pyomo_model.neural_network = OmltBlock()

# Transformer notre réseau de neurones en formulation Omlt pour l'utiliser comme expression Pyomo
formulation = FullSpaceSmoothNNFormulation(net)
pyomo_model.neural_network.build_formulation(formulation)

In [9]:

# Première sortie du réseau de neurones : la traînée
f1 = pyomo_model.neural_network.outputs[0]
# Deuxième sortie du réseau de neurones : la portance
f2 = pyomo_model.neural_network.outputs[1]

# 3. Formuler l'objectif de maximisation de f1 - f2
pyomo_model.objective = pyo.Objective(expr=f2 - f1, sense=pyo.maximize)

In [10]:

# 4. Ajouter des contraintes sur les valeurs de x et y
pyomo_model.con_x = pyo.Constraint(expr=pyomo_model.neural_network.inputs[0] >= 0)
pyomo_model.con_y = pyo.Constraint(expr=pyomo_model.neural_network.inputs[1] >= 0)

In [ ]:


solver = pyo.SolverFactory("ipopt")
status = solver.solve(pyomo_model, tee=False)

In [12]:
# 5. Afficher les résultats optimaux
optimal_x = pyo.value(pyomo_model.neural_network.inputs[0])
optimal_y = pyo.value(pyomo_model.neural_network.inputs[1])
optimal_f1 = pyo.value(f1)
optimal_f2 = pyo.value(f2)
optimal_value = optimal_f2 - optimal_f1

print(f'Valeurs optimales : x = {optimal_x}, y = {optimal_y}')
print(f'f1 = {optimal_f1}, f2 = {optimal_f2}')
print(f'Valeur de f1 - f2 maximale : {optimal_value}')

Valeurs optimales : x = 2.6682962680897364, y = 3.459258743014946
f1 = 0.624449062400395, f2 = 1.4618825961667332
Valeur de f1 - f2 maximale : 0.8374335337663382
